## Compile a Movidius Model

```sh
mvNCCompile model-ckpt-445500.meta \
    [-w .ckpt] \
    -s 12 \
    -in mlp_model/input \
    -on mlp_model/output/output \
    -o mnist_mlp_v1.graph
```

## Use a Compiled Model

In [1]:
from mvnc import mvncapi as mvnc
import os
import numpy
import cv2
import matplotlib.pyplot as plt
import time
import random

In [3]:
tmp_dir = os.path.join("../../../devops/tmp")

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join(tmp_dir,'MNIST_data'), one_hot=True)

/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../../devops/tmp/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../../devops/tmp/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../../devops/tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../../devops/tmp/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
graph_filename = '../tmp/mnist_mlp_v1.graph'
assert os.path.exists(graph_filename), "The model was not found."

In [7]:
devices = mvnc.EnumerateDevices()
if len(devices) == 0:
    print('No devices found')
    quit()

device = mvnc.Device(devices[0])

In [8]:
#Load graph
with open(graph_filename, mode='rb') as f:
    graphfile = f.read()

In [9]:
def getRandomImg():
    idx = random.randint(0, mnist.test.images.shape[0])
    imgs = mnist.test.images[idx,:]
    labels = mnist.test.labels[idx]
    return imgs, labels

In [10]:
try:
    imgs, labels = getRandomImg()
    
    # connect to the device and load the graph
    device.OpenDevice()
    graph = device.AllocateGraph(graphfile)

    # load the data into the graph
    # type must be assign as float16
    graph.LoadTensor(imgs.astype(numpy.float16), 'user object')
    output, userobj = graph.GetResult()

    # show the result
    print(output)
    print(numpy.argmax(output), numpy.argmax(labels))

    # close the device
    graph.DeallocateGraph()
except Exception as e:
    print(e)
finally:
    device.CloseDevice()
    print('Finished')    

[ 0.    0.   17.44  0.    0.    0.    0.    0.    0.    0.  ]
2 2
Finished
